In [51]:
import os
import cv2
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader, random_split
import albumentations as A
from albumentations.pytorch import ToTensorV2
import segmentation_models_pytorch as smp
from tqdm import tqdm

In [52]:
class BrainMRIDataset(Dataset):
    def __init__(self, image_dir, mask_dir, augment=False):
        self.image_dir = image_dir
        self.mask_dir = mask_dir
        self.images = sorted(os.listdir(image_dir))
        self.masks = sorted(os.listdir(mask_dir))

        self.transform = A.Compose([
            A.Resize(256, 256),
            A.HorizontalFlip(p=0.5),
            A.RandomRotate90(p=0.5),
            A.ShiftScaleRotate(p=0.2),
            A.Normalize(mean=(0.5,), std=(0.5,)),
            ToTensorV2()
        ]) if augment else A.Compose([
            A.Resize(256, 256),
            A.Normalize(mean=(0.5,), std=(0.5,)),
            ToTensorV2()
        ])

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        img_path = os.path.join(self.image_dir, self.images[idx])
        mask_path = os.path.join(self.mask_dir, self.masks[idx])

        # load
        image = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
        mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)

        # resize
        image = cv2.resize(image, (256, 256))
        mask = cv2.resize(mask, (256, 256), interpolation=cv2.INTER_NEAREST)

        # apply augmentations
        augmented = self.transform(image=image, mask=mask)
        image = augmented["image"]
        mask = augmented["mask"]

        # convert to tensors
        image = image.float() / 255.0  # [1,H,W]

        # map mask to 0..3
        mask = (mask // 85).long()

        return image, mask



In [53]:
def get_model():
    model = smp.Unet(
    encoder_name="resnet18",
    encoder_weights="imagenet",
    in_channels=1,
    classes=4
    )
    return model

In [54]:
def dice_loss(pred, target, num_classes=4, smooth=1e-5):
    pred = torch.softmax(pred, dim=1)   # [B,C,H,W]
    target_onehot = torch.nn.functional.one_hot(target, num_classes).permute(0,3,1,2).float()

    dims = (0,2,3)
    intersection = (pred * target_onehot).sum(dims)
    cardinality = pred.sum(dims) + target_onehot.sum(dims)
    dice = (2. * intersection + smooth) / (cardinality + smooth)
    return 1 - dice.mean()

def dice_score(outputs, targets, num_classes=4, epsilon=1e-7):
    """ Multi-class Dice score (macro average). """
    outputs = torch.softmax(outputs, dim=1)      # [B, C, H, W]
    preds = torch.argmax(outputs, dim=1)         # [B, H, W]

    dice_per_class = []
    for cls in range(num_classes):
        pred_cls = (preds == cls).float()
        true_cls = (targets == cls).float()

        intersection = (pred_cls * true_cls).sum()
        union = pred_cls.sum() + true_cls.sum()

        dice = (2 * intersection + epsilon) / (union + epsilon)
        dice_per_class.append(dice.item())

    return sum(dice_per_class) / num_classes


def iou_score(outputs, targets, num_classes=4, epsilon=1e-7):
    """ Multi-class IoU (macro average). """
    outputs = torch.softmax(outputs, dim=1)
    preds = torch.argmax(outputs, dim=1)

    iou_per_class = []
    for cls in range(num_classes):
        pred_cls = (preds == cls).float()
        true_cls = (targets == cls).float()

        intersection = (pred_cls * true_cls).sum()
        union = pred_cls.sum() + true_cls.sum() - intersection

        iou = (intersection + epsilon) / (union + epsilon)
        iou_per_class.append(iou.item())

    return sum(iou_per_class) / num_classes

ce_loss = nn.CrossEntropyLoss()

def ce_dice_loss(pred, target):
    return ce_loss(pred, target) + dice_loss(pred, target)


In [55]:
def train_model(model, modality, train_loader, val_loader, optimizer, epochs, device, best_loss=1):
    history = {"train_loss": [], "val_loss": [], "val_dice": [], 'val_iou': []}

    for epoch in range(epochs):
        # ---- Training ----
        model.train()
        running_loss = 0.0
        with tqdm(total=len(train_loader), desc=f"Epoch {epoch+1}/{epochs}", unit="batch") as pbar:
            for imgs, masks in train_loader:
                imgs, masks = imgs.to(device), masks.to(device)

                optimizer.zero_grad()
                outputs = model(imgs)
                loss = ce_dice_loss(outputs, masks)
                loss.backward()

                optimizer.step()

                running_loss += loss.item()
                pbar.update(1)

        avg_train_loss = running_loss / len(train_loader)

        # ---- Validation ----
        model.eval()
        val_loss, val_dice, val_iou = 0.0, 0.0, 0.0
        with torch.no_grad():
            for imgs, masks in val_loader:
                imgs, masks = imgs.to(device), masks.to(device)
                outputs = model(imgs)

                loss = ce_dice_loss(outputs, masks)
                val_loss += loss.item()
                val_dice += dice_score(outputs, masks)
                val_iou += iou_score(outputs, masks)

        avg_val_loss = val_loss / len(val_loader)
        avg_val_dice = val_dice / len(val_loader)
        avg_val_iou  = val_iou / len(val_loader)

        history["train_loss"].append(avg_train_loss)
        history["val_loss"].append(avg_val_loss)
        history["val_dice"].append(avg_val_dice)
        history["val_iou"].append(avg_val_iou)

        print(f"Epoch {epoch+1}/{epochs} "
              f"- Train Loss: {avg_train_loss:.4f} "
              f"- Val Loss: {avg_val_loss:.4f} "
              f"- Val Dice: {avg_val_dice:.4f} "
              f"- Val IoU: {avg_val_iou:.4f}")


        if avg_val_loss < best_loss:
            best_loss = avg_val_loss
            save_path = os.path.join("..", "weights", f"{modality}_unet_resnet18.pth")
            torch.save(model.state_dict(), save_path)


    return history

In [56]:
def train_one_modality(modality, root_dir="../data/dataset", epochs=10, batch_size=8, lr=1e-4, device="cuda"):
    img_dir = os.path.join(root_dir, modality, "images")
    mask_dir = os.path.join(root_dir, modality, "masks")

    dataset = BrainMRIDataset(img_dir, mask_dir, augment=True)
    val_size = int(0.2 * len(dataset))
    train_size = len(dataset) - val_size
    train_ds, val_ds = random_split(dataset, [train_size, val_size])

    train_loader = DataLoader(train_ds, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_ds, batch_size=batch_size, shuffle=False)

    model = get_model().to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)

    return train_model(model, modality, train_loader, val_loader, optimizer, epochs, device)


In [57]:
histories = {}
modalities = ["flair", "t1", "t1ce", "t2"]

for m in modalities:
    hist = train_one_modality(m, epochs=20, batch_size=8, lr=1e-4)
    histories[m] = hist
    print(f"\nfinished training {m}\n")

Epoch 1/20: 100%|██████████| 543/543 [00:50<00:00, 10.80batch/s]


Epoch 1/20 - Train Loss: 1.0139 - Val Loss: 0.7081 - Val Dice: 0.4804 - Val IoU: 0.3970


Epoch 2/20: 100%|██████████| 543/543 [00:53<00:00, 10.16batch/s]


Epoch 2/20 - Train Loss: 0.6254 - Val Loss: 0.5949 - Val Dice: 0.5118 - Val IoU: 0.4207


Epoch 3/20: 100%|██████████| 543/543 [00:52<00:00, 10.35batch/s]


...

Epoch 18/20 - Train Loss: 0.4473 - Val Loss: 0.4897 - Val Dice: 0.5838 - Val IoU: 0.4842


Epoch 19/20: 100%|██████████| 544/544 [00:53<00:00, 10.23batch/s]


Epoch 19/20 - Train Loss: 0.4411 - Val Loss: 0.4603 - Val Dice: 0.6150 - Val IoU: 0.5137


Epoch 20/20: 100%|██████████| 544/544 [00:51<00:00, 10.47batch/s]


Epoch 20/20 - Train Loss: 0.4356 - Val Loss: 0.4668 - Val Dice: 0.6111 - Val IoU: 0.5108

finished training t2

